In [ ]:
import sys, os, glob
import pandas as pd
import numpy as np
import pathlib
import csv 
from PIL import Image
from resizeimage import resizeimage
import random

In [ ]:
rootPath = '/IMFDB'
IMFDB_final_path = rootPath + 'IMFDB_final'
IMFDB_CSV_path = rootPath + 'IMFDB_CSV'
IMFDB_final = pathlib.Path(IMFDB_final_path)

In [ ]:
emotionsNumber = { "ANGER":0 , "DISGUST":1 , "FEAR":2 , "HAPPINESS":3 , "SADNESS":4 , "SURPRISE":5 , "NEUTRAL":6  }
def getEmotionNumber(emotion):
  try:z
    return emotionsNumber[emotion]
  except:
    return -1

In [ ]:
def fileExits(path):
  my_file = pathlib.Path(path)
  if my_file.is_file():
    return True
  else:
    return False

In [ ]:
def isDir(path):
  my_file = pathlib.Path(path)
  if my_file.is_dir():
    return True
  else:
    return False

In [ ]:
def getFilePath(moviePath,actorName,movieName):
  try:
    os.chdir(moviePath)
    for file in glob.glob("*.txt"):
      return file
  except:
    print("Text File Not Found -- Ignored")
    return False

In [ ]:
def checkCSVCreated(ActorName):
  csvPath = IMFDB_CSV_path + '/' + ActorName + ".csv"
  return fileExits(csvPath)

In [ ]:
def getEmotionsFromFile(path):
  file = open(filePath, 'r') 
  Lines = file.readlines()
  emotions = {}
  for line in Lines:
    keys = line.split("\t")
    if(len(keys)>11):
      emotions[keys[2]] = keys[11]
  return emotions

In [ ]:
def getResizeImage(path):
  try:
    fd_img = open(path, 'rb')
    img = Image.open(fd_img).convert('LA')
    img = resizeimage.resize_cover(img, [48, 48])
    fd_img.close()
    return img
  except:
    print("Image Very Small -- Ignored")
    return False

In [ ]:
def getRandomTag():
  if random.uniform(0, 1) < 0.3 :
    return "PublicTest"
  else :
    return "Training"

In [ ]:
def createCSV(dataArray,actorName):
  if len(dataArray) == 0:
    print("Data Missing for **" + actorName + "** -- Ignored")
    return
  csvPath = IMFDB_CSV_path + '/' + actorName + ".csv"
  data = np.array(dataArray)
  dataset = pd.DataFrame({'emotion':data[:,0] , 'pixels': data[:, 1], 'Usage': data[:, 2]})
  dataset.to_csv(csvPath, index = False)

In [ ]:
for actor in IMFDB_final.glob('*'):
  dataArray = []
  if checkCSVCreated(actor.name): continue
  print("start -- " + actor.name)
  for movie in actor.glob('*'):
    moviePath = IMFDB_final_path + "/" + actor.name + "/" + movie.name
    if not isDir(moviePath): continue
    imagesPath = moviePath + "/images/"
    filePath = getFilePath(moviePath,actor.name,movie.name)
    if not filePath: continue
    emotions = getEmotionsFromFile(filePath)
    for key in emotions:
      emotionNo = getEmotionNumber(emotions[key])
      if emotionNo == -1: continue
      imagePath = imagesPath + key
      print(imagePath)
      img = getResizeImage(imagePath)
      if not img: continue
      imgArrayFull = np.asarray(img).flatten()
      imgArray = imgArrayFull[::2]
      pixel_list = imgArray.tolist()
      pixel_str_list = map(str, pixel_list)
      img_str = ' '.join(pixel_str_list)
      data = []
      data.append(emotionNo)
      data.append(img_str)
      data.append("Training")
      dataArray.append(data)
  createCSV(dataArray,actor.name)

In [ ]:
os.chdir(IMFDB_CSV_path)
frames = []
for file in glob.glob("*.csv"):
  filePath = IMFDB_CSV_path + "/" + file
  df =  pd.read_csv(filePath)
  frames.append(df)
df = pd.concat(frames)

In [ ]:
df.info()

In [ ]:
allFilePath = IMFDB_CSV_path + "/IMFDB.csv" 
df.to_csv(allFilePath, index = False)

In [ ]:
data = np.array(dataArray)
dataset = pd.DataFrame({'emotion':data[:,0] , 'pixels': data[:, 1], 'Usage': data[:, 2]})
dataset.to_csv(r'IMFDB/IMFDB.csv', index = False)
#end